In [2]:
import numpy as np
from math import pi
from gdshelpers.geometry.chip import Cell
from gdshelpers.parts.waveguide import Waveguide
from gdshelpers.parts.coupler import GratingCoupler
from gdshelpers.parts.resonator import RingResonator
from gdshelpers.parts.splitter import Splitter
from gdshelpers.layout import GridLayout
from gdshelpers.parts.marker import CrossMarker
from gdshelpers.parts.marker import SquareMarker
from gdshelpers.helpers.positive_resist import convert_to_positive_resist
from gdshelpers.parts.port import Port
from shapely.geometry import Polygon
from shapely.affinity import scale
from shapely.affinity import translate
from shapely.affinity import rotate
from gdshelpers.geometry import geometric_union
from gdshelpers.helpers.under_etching import create_holes_for_under_etching
from gdshelpers.geometry.ebl_frame_generators import raith_marker_frame
from gdshelpers.parts.text import Text
from shapely.geometry import Point
import importlib
import aco_components
importlib.reload(aco_components)

<module 'aco_components' from '/Users/haoqindeng/Desktop/MoLiGroup/IDT_gds/aco_components.py'>

In [47]:
grating_pitch = [0.670, 0.675, 0.680, 0.685, 0.690, 0.695, 0.700, 0.705, 0.710, 0.715, 0.720, 0.725, 0.730]
maximum_duty = [0.840, 0.835, 0.840, 0.845,  0.850,  0.855, 0.860, 0.865, 0.870]

coupler1_params = {
        'width': 1.2,
        'full_opening_angle': np.deg2rad(40),  # 40
        'grating_period': 0.7,
        'grating_ff': 0.85,  # minigap = 30nm
        # 'ap_max_ff':0.85,
        'ap_max_ff':0.85,
        'n_gratings': 40,  # 20
        'taper_length': 16,  # 16um
        'n_ap_gratings': 0,  # 20
}


In [48]:
def make_grating_coupler(cell, coupler_params, offset=(0,0)):
    gc1 = GratingCoupler.make_traditional_coupler(origin=offset, **coupler_params)
    wg1 = Waveguide.make_at_port(gc1.port)
    wg1.add_straight_segment(length=50)
    wg1.add_bend(radius=50, angle=-np.pi/2)
    wg1.add_straight_segment(length=100)

    gc2 = GratingCoupler.make_traditional_coupler(origin=(offset[0] + 250, offset[1]), **coupler_params)
    wg2 = Waveguide.make_at_port(gc2.port)
    wg2.add_straight_segment(length=50)
    wg2.add_bend(radius=50, angle=np.pi/2)
    wg2.add_straight_segment(length=100)

    cell.add_to_layer(1, convert_to_positive_resist( [gc1, gc2, wg1, wg2],  buffer_radius=5))
    # cell.add_to_layer(1, gc1, gc2, wg1, wg2)
    return cell

cell = Cell('GCSWP')
for i in range(len(maximum_duty)):
    for j in range(len(grating_pitch)):
        offset = (i*320, j*170)
        coupler1_params['grating_period'] = grating_pitch[j]
        coupler1_params['grating_ff'] = maximum_duty[i]
        # coupler1_params['ap_max_ff'] = maximum_duty[i]

        cell = make_grating_coupler(cell, coupler_params=coupler1_params, offset=offset)



# cell = make_grating_coupler(cell, coupler_params=coupler1_params, offset=(40,30))
cell.show()

In [49]:
cell.save('SOS_Char/gc_sweep.gds')